<a href="https://colab.research.google.com/github/cristhiansito123/mai_hand/blob/main/scripts/training/model_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Llamado de los landsmark

In [2]:
import json
import requests
import os
import shutil
import time
import sys

import numpy as np
import pandas as pd

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
%matplotlib inline

In [4]:
train_path = '/content/drive/MyDrive/signal_language/asl_dataset'


In [5]:
metadata_path = train_path+'/landmarks.csv'

metadata = pd.read_csv(metadata_path, sep=',')

In [6]:
metadata

,landsmark_1,landsmark_2,landsmark_3
0,0,NaN,NaN
1,0.4666137993335724,0.846637,0.000001
2,0.5910995006561279,0.649522,0.023281
3,0.6541032791137695,0.455880,0.004872
4,0.7205207347869873,0.309535,-0.039252
...,...,...,...
47075,0.5599057674407959,0.602437,-0.124475
47076,0.35630279779434204,0.644029,-0.070245
47077,0.3153184652328491,0.544756,-0.158190
47078,0.42366790771484375,0.594285,-0.145559


In [7]:
len(metadata)

47080

In [8]:
labels = []
landsmark = []
for i in range(int(len(metadata)/22)):
  labels.append(metadata.iloc[22*i]['landsmark_1'])
  temp = []
  for j in range(21):
    temp.append(metadata.iloc[22*i+j+1].values)
  landsmark.append(temp)


In [9]:
landsmark

[[array(['0.4666137993335724', 0.846636950969696, 1.1634530210358207e-06],
        dtype=object),
  array(['0.5910995006561279', 0.649522066116333, 0.0232811942696571],
        dtype=object),
  array(['0.6541032791137695', 0.4558798670768738, 0.0048720352351665],
        dtype=object),
  array(['0.7205207347869873', 0.309535264968872, -0.0392524190247058],
        dtype=object),
  array(['0.7134832739830017', 0.1527848094701767, -0.0807405412197113],
        dtype=object),
  array(['0.3659366965293884', 0.3013232350349426, 0.0708740577101707],
        dtype=object),
  array(['0.4030618667602539', 0.0754225701093673, 0.0086949318647384],
        dtype=object),
  array(['0.542731523513794', 0.0576635301113128, -0.0509725473821163],
        dtype=object),
  array(['0.6345841884613037', 0.1129290908575058, -0.0833439007401466],
        dtype=object),
  array(['0.30883532762527466', 0.3212186098098755, 0.0137973241508007],
        dtype=object),
  array(['0.3766554892063141', 0.038998529314

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
landsmark = np.asarray(landsmark)

In [12]:
landsmark = landsmark.reshape(landsmark.shape[0], landsmark.shape[1]*landsmark.shape[2])

In [13]:
labels = np.asarray(labels)

In [17]:
unique_values, value_counts = np.unique(labels, return_counts=True)

classes = unique_values[value_counts > 1]

Definir modelo

In [66]:
import tensorflow as tf




Convertir arreglo a numérico para hallar las labels a números

In [29]:
classes

array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c',
       'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p',
       'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'], dtype='<U1')

In [30]:
def convertir_a_numerico(arreglo):
    arreglo_numerico = []
    for elemento in arreglo:
        if elemento.isdigit():  # Verificar si el elemento es un dígito
            arreglo_numerico.append(int(elemento))
        else:
            # Verificar si el elemento es una letra y convertirla a valor numérico
            valor_numerico = ord(elemento.upper()) - ord('A') + 10
            arreglo_numerico.append(valor_numerico)
    return arreglo_numerico

In [31]:
classes_n = convertir_a_numerico(classes)

Conversión de características

In [33]:
def convertir_a_letra(numero):
    if numero >= 0 and numero <= 9:
        return str(numero)
    elif numero >= 10 and numero <= 35:
        return chr(ord('A') + numero - 10)
    else:
        raise ValueError("El número está fuera del rango permitido")

In [34]:
def convertir_a_numero(letra):
    if letra.isdigit():
        return int(letra)
    elif len(letra) == 1 and letra.isalpha():
        return ord(letra.upper()) - ord('A') + 10
    else:
        raise ValueError("El valor no es una letra o número válido")

In [41]:
labels_n = []
for i in range(len(labels)):
  labels_n.append(convertir_a_numero(labels[i]))

In [47]:
landsmark.shape

(2140, 63)

In [50]:
labels_n = np.asarray(labels_n)

In [52]:
X_train, X_val, y_train, y_val = train_test_split(landsmark, labels_n, test_size=0.2, random_state=42)

In [62]:
from tensorflow.keras.utils import to_categorical

# Convierte las etiquetas de clase a codificación one-hot
y_train_one_hot = to_categorical(y_train)
y_val_one_hot = to_categorical(y_val)

In [75]:
def entrenamiento(X_train, y_train_one_hot, X_val, y_val_one_hot,learning_rate,batch_size):
    # Definir la arquitectura del modelo
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(63,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(len(classes), activation='softmax')
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    # Compilar el modelo
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])

    # Entrenar el modelo
    model.fit(X_train, y_train_one_hot, batch_size=batch_size, epochs=200, validation_data=(X_val, y_val_one_hot))

In [76]:
entrenamiento(X_train.astype(np.float32), y_train_one_hot, X_val.astype(np.float32), y_val_one_hot, 0.001,32)


Epoch 1/200
54/54 [==============================] - 1s 4ms/step - loss: 3.4758 - accuracy: 0.0940 - val_loss: 3.3159 - val_accuracy: 0.1332
Epoch 2/200
54/54 [==============================] - 0s 2ms/step - loss: 3.0712 - accuracy: 0.2395 - val_loss: 2.7182 - val_accuracy: 0.4065
Epoch 3/200
54/54 [==============================] - 0s 2ms/step - loss: 2.3878 - accuracy: 0.4357 - val_loss: 2.0170 - val_accuracy: 0.5864
Epoch 4/200
54/54 [==============================] - 0s 2ms/step - loss: 1.7945 - accuracy: 0.5660 - val_loss: 1.5344 - val_accuracy: 0.6822
Epoch 5/200
54/54 [==============================] - 0s 2ms/step - loss: 1.3969 - accuracy: 0.6595 - val_loss: 1.2331 - val_accuracy: 0.7220
Epoch 6/200
54/54 [==============================] - 0s 2ms/step - loss: 1.1392 - accuracy: 0.7231 - val_loss: 1.0180 - val_accuracy: 0.7570
Epoch 7/200
54/54 [==============================] - 0s 2ms/step - loss: 0.9653 - accuracy: 0.7704 - val_loss: 0.8901 - val_accuracy: 0.7360
Epoch 8/200
5